In [ ]:
if mag_rtn_4sa is not None:
        ## Defining & Calculating Variables
    #==============================================================
    # 🧲 MAG_RTN_4SA Data Preparation
    #==============================================================

    # Get magnetic field and time data for RTN coordinates at 4 samples per cycle
    mag_data_RTN_4sa = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')
    mag_time_RTN_4sa, mag_field_RTN_4sa = mag_data_RTN_4sa.times, mag_data_RTN_4sa.y

        # Access components of magnetic field in RTN coordinates at 4 samples per second
    split_vec('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')
    br_RTN_4sa = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc_x')
    bt_RTN_4sa = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc_y')
    bn_RTN_4sa = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc_z')
    bmag_RTN_4sa = np.sqrt(br_RTN_4sa.y**2 + bt_RTN_4sa.y**2 + bn_RTN_4sa.y**2)

    br_RTN_4sa = br_RTN_4sa.y
    bt_RTN_4sa = bt_RTN_4sa.y
    bn_RTN_4sa = bn_RTN_4sa.y

        # Converting the magnetic field time to a timezone-aware datetime object
    datetime_mag_RTN_4sa = np.array([dt.replace(tzinfo=timezone.utc) for dt in time_datetime(mag_time_RTN_4sa)])

        # Calculate magnetic pressure
    mu_0 = constants.mu_0  # Permeability of free space
    pmag_RTN_4sa = (bmag_RTN_4sa**2) / (2 * mu_0)

        # Convert to nPa (nanoPascals) to match your other pressure units
    pmag_RTN_4sa = pmag_RTN_4sa * 1e9

In [ ]:
if mag_rtn is not None:
        # # Get magnetic field and time data for RTN coordinates at full resolution
    mag_data_RTN = get_data('psp_fld_l2_mag_RTN')
    mag_time_RTN, mag_field_RTN = mag_data_RTN.times, mag_data_RTN.y

    # # Access components of magnetic field in RTN coordinates at full resolution
    split_vec('psp_fld_l2_mag_RTN')
    br_RTN = get_data('psp_fld_l2_mag_RTN_x')
    bt_RTN = get_data('psp_fld_l2_mag_RTN_y')
    bn_RTN = get_data('psp_fld_l2_mag_RTN_z')
    bmag_RTN = np.sqrt(br_RTN.y**2 + bt_RTN.y**2 + bn_RTN.y**2)

    br_RTN = br_RTN.y
    bt_RTN = bt_RTN.y
    bn_RTN = bn_RTN.y

    # # Converting the magnetic field time to a timezone-aware datetime object
    datetime_mag_RTN = np.array([dt.replace(tzinfo=timezone.utc) for dt in time_datetime(mag_time_RTN)])
    
    mu_0 = constants.mu_0  # Permeability of free space
    # # Calculate magnetic pressure
    pmag_RTN = (bmag_RTN**2) / (2 * mu_0)

    # # Convert to nPa (nanoPascals) to match your other pressure units
    pmag_RTN = pmag_RTN * 1e9

In [ ]:
if spe_sf0_pad is not None:
            #==============================================================
    # Strahl Data Preparation (PAD)
    #==============================================================

    #Access electron strahl pitch-angle distribution (PAD)
    #epad_data is a data structure
    epad_data = get_data('psp_spe_EFLUX_VS_PA_E')
    epad_PA = get_data('psp_spe_PITCHANGLE')

    #access the times by doing whatever you named it.times
    epad_times=epad_data.times

    #values are always stored in whatever you named it.y
    epad_vals = epad_data.y

    #Pulling in the epad_PA values by adding "whatever you named it".y
    epad_PA_vals = epad_PA.y

    #Note for E1-E9, electron strahl energy bin is 8, for E10 and above, it's 12
    #strahl_energy_index = ebin
        #energy_index = strahl_energy_index
    if (time_double(trange[0]) < time_double('2021-11-15')):
        strahl_energy_index = 8
    else:
        strahl_energy_index = 10
    
    energy_index = strahl_energy_index

    #This takes whatever energy slice we're looking at and accesses that flux information
    #This is a 3D data cube
    #The ":" means take ALL of that row and ALL of that column, and we're taking that whole energy plane
    epad_strahl = epad_vals[:,:,energy_index]


    #--------------------------------------------------------------------------------------------------------------------------\\
    # Convert Unix timestamps to timezone-aware datetime objects in UTC

    # Most space science data records time as Unix timestamps. A Unix timestamp is the number of seconds 
    # that have elapsed since January 1, 1970 (also known as the "epoch"). These timestamps are in Coordinated Universal Time (UTC) 
    # but are considered "naive" datetime objects in Python because they lack explicit timezone information.

    # To work with these timestamps effectively, especially when combining data from different sources or performing timezone-aware calculations, 
    # it's important to convert them into timezone-aware datetime objects.

    # Here's what the code does:
    # 1. Use 'time_datetime(epad_times)' to convert the array of Unix timestamps 'epad_times' into Python datetime objects.
    #    This function returns a list of naive datetime objects (without timezone information).
    # 2. Use a list comprehension to iterate over each datetime object 'dt' in the list:
    #    - For each 'dt', call 'dt.replace(tzinfo=timezone.utc)' to assign the UTC timezone.
    #    - This creates a new datetime object that is timezone-aware.
    # 3. Convert the list of timezone-aware datetime objects into a NumPy array using 'np.array()' for efficient numerical operations.

    # The result is 'datetime_spe', a NumPy array of timezone-aware datetime objects in UTC.
    datetime_spe = np.array([dt.replace(tzinfo=timezone.utc) for dt in time_datetime(epad_times)])

    #--------------------------------------------------------------------------------------------------------------------------//

    # Prepare the time array to match the dimensions required by pcolormesh.
    # pcolormesh requires the X (time rows), Y (pitch angle columns), and color (flux) arrays to have the same shape.
    # Our flux data 'epad_strahl' has shape (number of time steps, number of pitch angle bins).
    # We expand 'datetime_spe' from shape (N,) to (N, 1) to make it a 2D column vector.

    # This creates a column vector with N rows (where N is the number of time steps) and 1 column.
    # Then, we repeat each time value across all pitch angle bins to create 'times_spe_repeat' with shape (N, M).
    # This aligns the time array with the flux data for proper plotting.

    # i.e., We create 'times_spe_repeat' by repeating 'datetime_spe' across pitch angle bins.
    # This creates a new array with the same number of columns as the number of pitch angle bins.

    #Pro-Tip, in python matrices are indexed as ⭐️ (rows, columns) ⭐️

    # So, starting with (e.g.):
    # [T1, T2, T3]  # A 1D array (row vector) of timestamps

    # We expand it to (N, 1): <--- (using np.expand_dims) 
    # [[T1],
    #  [T2],
    #  [T3]]  # A 2D column vector

    # We then repeat each of these values across all pitch angle bins to get: <--- (using np.repeat) (Time steps, 12 PA bins)
    # [T1, T1, T1, T1, T1, T1, T1, T1, T1, T1, T1, T1]  <--- 12 PA Bins for Time step 1
    # [T2, T2, T2, T2, T2, T2, T2, T2, T2, T2, T2, T2]  <--- 12 PA Bins for Time step 2
    # [T3, T3, T3, T3, T3, T3, T3, T3, T3, T3, T3, T3]  <--- 12 PA Bins for Time step 3
    # ... <--- (continues for all time steps)

    # This creates a 2D array where each row corresponds to a timestamp and each column represents a pitch angle bin.

    times_spe_repeat = np.repeat(
        np.expand_dims(datetime_spe, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        epad_strahl.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )

    # Apply logarithmic transformation to the electron PAD data
    log_epad_strahl = np.log10(epad_strahl)

    #compute centroids
    centroids_epad = np.ma.average(epad_PA_vals, 
                        weights=epad_strahl, 
                        axis=1)

In [ ]:
if spi_sf00_mom is not None:
    #==============================================================
    # Proton Calculations
    #==============================================================
    
    #Proton temperature anisotropy from Temperature Tensor

    #Access tensor elements -- The temperature is an array of 9 elements. We want to find out how much temp is aligned parallel or perp to the mag field.
    T_Tens = get_data('psp_spi_sf00_T_TENSOR_INST') #Obtains the values from the tensor array
    T_XX = T_Tens.y[:,0]
    T_YY = T_Tens.y[:,1]
    T_ZZ = T_Tens.y[:,2]
    T_XY = T_Tens.y[:,3]
    T_XZ = T_Tens.y[:,4]
    T_YZ = T_Tens.y[:,5]

    T_YX = T_XY
    T_ZX = T_XZ
    T_ZY = T_YZ

    #Access magnetic field in span-I coordinates
    B_spi = get_data('psp_spi_sf00_MAGF_INST')
    B_X = B_spi.y[:,0]
    B_Y = B_spi.y[:,1]
    B_Z = B_spi.y[:,2]
    B_mag_XYZ = np.sqrt(B_X**2 + B_Y**2 + B_Z**2)

    #Project Tensor onto B field, find perpendicular and parallel components
    T_parallel=[]
    T_perpendicular=[]
    Anisotropy=[]
    for i in range(len(B_X)):  #Calculates Tperp and Tpar from the projection of the magnetic field vector
        Sum_1=B_X[i]*B_X[i]*T_XX[i]
        Sum_2=B_X[i]*B_Y[i]*T_XY[i]
        Sum_3=B_X[i]*B_Z[i]*T_XZ[i]
        Sum_4=B_Y[i]*B_X[i]*T_YX[i]
        Sum_5=B_Y[i]*B_Y[i]*T_YY[i]
        Sum_6=B_Y[i]*B_Z[i]*T_YZ[i]
        Sum_7=B_Z[i]*B_X[i]*T_ZX[i]
        Sum_8=B_Z[i]*B_Y[i]*T_ZY[i]
        Sum_9=B_Z[i]*B_Z[i]*T_ZZ[i]    
        T_para=((Sum_1+Sum_2+Sum_3+Sum_4+Sum_5+Sum_6+Sum_7+Sum_8+Sum_9)/(B_mag_XYZ[i])**2)
        Trace_Temp=(T_XX[i]+T_YY[i]+T_ZZ[i])
        T_perp=(Trace_Temp-T_para)/2.0
        T_parallel.append((Sum_1+Sum_2+Sum_3+Sum_4+Sum_5+Sum_6+Sum_7+Sum_8+Sum_9)/(B_mag_XYZ[i])**2)
        T_perpendicular.append(T_perp)
        Anisotropy.append(T_perp/T_para)

    #Proton Analysis: Velocity, Energy Flux, and Temperature Anisotropy
    #==============================================================

    # This section performs the following operations on the proton data:
    # 1. Extracts and calculates velocity components (Vr, Vt, Vn) and magnitude
    # 2. Accesses proton energy flux data for energy, theta, and phi
    # 3. Extracts times, energy flux, and energy values
    # 4. Computes centroids for energy, theta, and phi
    # 5. Prepares time arrays for plotting
    # 6. Calculates temperature anisotropy from temperature tensor and magnetic field data
    # Note: Commented out code for interpolation to sf00 fit file cadence

    #Access components of velocity
    split_vec('psp_spi_sf00_VEL_RTN_SUN') #split_vec saparates the three components out, we're in RTN so "x" becomes our "r" and so on...
    vr_ = get_data('psp_spi_sf00_VEL_RTN_SUN_x')
    vr = vr_.y
    vt_ = get_data('psp_spi_sf00_VEL_RTN_SUN_y')
    vt = vt_.y
    vn_ = get_data('psp_spi_sf00_VEL_RTN_SUN_z')
    vn = vn_.y
    vmag = np.sqrt(vr**2 + vt **2 + vn**2) #Here we're defining the magnitude of this vector, the data is stored in the .y component

    #In the future we'll create a function that computes the magnitude.

    #access proton energy flux vs energy -- For the ions we want to look at the energy, theta and phi planes...
    spi_eflux_v_energy = get_data('psp_spi_sf00_EFLUX_VS_ENERGY')
    spi_eflux_v_theta = get_data('psp_spi_sf00_EFLUX_VS_THETA')
    spi_eflux_v_phi = get_data('psp_spi_sf00_EFLUX_VS_PHI')

    spi_times = spi_eflux_v_energy.times #extracting the times

    spi_nrg_flux = spi_eflux_v_energy.y #energy flux vs energy, this is the data that will go into the color bar

    # Apply logarithmic transformation to the spi nrg flux vs nrg data
    log_spi_nrg_flux = np.log10(spi_nrg_flux)

    spi_nrg_vals = spi_eflux_v_energy.v #energy flux vs the actual energy values, this is the y axis. / The energy values for the  32 channels coppied at every time step (they do not change)

    #When we're looking at eflux vs e, at each time step we have the amount of flux and at what energy did we measure that flux.
    #Energy vs phi, we have the amount of flux and at which phi value did we measure that flux.

    spi_nrg_flux_theta = spi_eflux_v_theta.y
    log_spi_nrg_flux_theta = np.log10(spi_nrg_flux_theta)
    spi_nrg_vals_theta = spi_eflux_v_theta.v
    spi_nrg_flux_phi = spi_eflux_v_phi.y
    log_spi_nrg_flux_phi = np.log10(spi_nrg_flux_phi)
    spi_nrg_vals_phi = spi_eflux_v_phi.v

    datetime_spi = time_datetime(spi_times)
    print(datetime_spi[0])
    print(datetime_spi[-1])

    times_spi_repeat = np.repeat(np.expand_dims(datetime_spi,1),32,1)
    times_spi_repeat_angle = np.repeat(np.expand_dims(datetime_spi,1),8,1) 

    #compute centroids
    centroids_spi_nrg = np.ma.average(spi_nrg_vals, 
                           weights=spi_nrg_flux, 
                           axis=1)
    centroids_spi_theta = np.ma.average(spi_nrg_vals_theta, 
                           weights=spi_nrg_flux_theta, 
                           axis=1)
    centroids_spi_phi = np.ma.average(spi_nrg_vals_phi, 
                           weights=spi_nrg_flux_phi, 
                           axis=1)


    # Proton Plasma Parameters Calculation
    #==============================================================
    # This cell calculates various plasma parameters for protons:
    # 1. Extracts plasma density and magnetic field data
    # 2. Calculates magnetic field magnitude
    # 3. Computes Alfvén speed
    # 4. Calculates parallel and perpendicular plasma beta
    # 5. Computes parallel and perpendicular plasma pressure
    # Note: All calculations use data from SPAN-Ion (SPI) instrument

    dens_spi = get_data('psp_spi_sf00_DENS') #the density of the plasma measured by span ion in units of cm^-3
    b_spi = get_data('psp_spi_sf00_MAGF_INST') #the magnetic field projected in the instrument frame at the same cadence as span ion

    # Calculate the magnitude of b_spi
    b_mag_spi = np.sqrt(np.sum(b_spi.y**2, axis=1))

    #The alfven speed
    v_alfven_spi = 21.8 * b_mag_spi/ np.sqrt(dens_spi.y) #alfven speed computed using spi density

    #The velocity of the solar wind
    v_sw_spi = np.sqrt(vr**2 + vt**2 + vn**2)

    # The Alfvén Mach number
    M_alfven_spi = v_sw_spi / v_alfven_spi

    #The Temperature
    temp_spi = get_data('psp_spi_sf00_TEMP')

    #The definition of beta is the particle pressure over the magnetic field pressure
    #p is for proton and this is the parallel and perpendicular beta
    beta_ppar_spi = (4.03E-11*dens_spi.y*T_parallel)/(1e-5*b_mag_spi)**2
    beta_pperp_spi = (4.03E-11*dens_spi.y*T_perpendicular)/(1e-5*b_mag_spi)**2
    beta_p_spi = (4.03E-11*dens_spi.y*temp_spi.y)/(1e-5*b_mag_spi)**2


    pressure_ppar_spi = 1.602E-4*dens_spi.y*T_parallel #the original number was 1.602E-13, this is the converstion factor for electron vots per cubic CM to pascals, then we multiplied it by 1E9 (1*10^9) to convert to nanopascals 
    pressure_pperp_spi = 1.602E-4*dens_spi.y*T_perpendicular
    pressure_p_spi = 1.602E-4*temp_spi.y*dens_spi.y 
    #Pperp over ppar eventually

    # Calculate magnetic pressure using Bmag_spi cadence (for tot pressure calc)
    mu_0 = constants.mu_0  # Permeability of free space
    pmag_spi = (b_mag_spi**2) / (2 * mu_0)
        # Convert to nPa (nanoPascals) to match your other pressure units
    pmag_spi = pmag_spi * 1e9
    #total pressure of magnetic plus proton (later add more terms like alpha and elec)
    pressure_tot_mag_p = pressure_p_spi + pmag_spi


    #dostance from sun
    sun_dist_ = get_data('psp_spi_sf00_SUN_DIST')
    sun_dist_km = sun_dist_.y
    sun_dist_rsun = sun_dist_km/695700.0


In [ ]:
if spi_sf0a_mom is not None:
    #==============================================================
    # Proton Calculations
    #==============================================================
    
    #Proton temperature anisotropy from Temperature Tensor

    #Access tensor elements -- The temperature is an array of 9 elements. We want to find out how much temp is aligned parallel or perp to the mag field.
    T_Tens_sf0a = get_data('psp_spi_sf0a_T_TENSOR_INST') #Obtains the values from the tensor array
    T_XX_sf0a = T_Tens_sf0a.y[:,0]
    T_YY_sf0a = T_Tens_sf0a.y[:,1]
    T_ZZ_sf0a = T_Tens_sf0a.y[:,2]
    T_XY_sf0a = T_Tens_sf0a.y[:,3]
    T_XZ_sf0a = T_Tens_sf0a.y[:,4]
    T_YZ_sf0a = T_Tens_sf0a.y[:,5]

    T_YX_sf0a = T_XY_sf0a
    T_ZX_sf0a = T_XZ_sf0a
    T_ZY_sf0a = T_YZ_sf0a

    #Access magnetic field in span-I coordinates
    B_spi_sf0a = get_data('psp_spi_sf0a_MAGF_INST')
    B_X_sf0a = B_spi_sf0a.y[:,0]
    B_Y_sf0a = B_spi_sf0a.y[:,1]
    B_Z_sf0a = B_spi_sf0a.y[:,2]
    B_mag_XYZ_sf0a = np.sqrt(B_X_sf0a**2 + B_Y_sf0a**2 + B_Z_sf0a**2)

    #Project Tensor onto B field, find perpendicular and parallel components
    T_parallel_sf0a=[]
    T_perpendicular_sf0a=[]
    Anisotropy_sf0a=[]
    for i in range(len(B_X_sf0a)):  #Calculates Tperp and Tpar from the projection of the magnetic field vector
        Sum_1_sf0a=B_X_sf0a[i]*B_X_sf0a[i]*T_XX_sf0a[i]
        Sum_2_sf0a=B_X_sf0a[i]*B_Y_sf0a[i]*T_XY_sf0a[i]
        Sum_3_sf0a=B_X_sf0a[i]*B_Z_sf0a[i]*T_XZ_sf0a[i]
        Sum_4_sf0a=B_Y_sf0a[i]*B_X_sf0a[i]*T_YX_sf0a[i]
        Sum_5_sf0a=B_Y_sf0a[i]*B_Y_sf0a[i]*T_YY_sf0a[i]
        Sum_6_sf0a=B_Y_sf0a[i]*B_Z_sf0a[i]*T_YZ_sf0a[i]
        Sum_7_sf0a=B_Z_sf0a[i]*B_X_sf0a[i]*T_ZX_sf0a[i]
        Sum_8_sf0a=B_Z_sf0a[i]*B_Y_sf0a[i]*T_ZY_sf0a[i]
        Sum_9_sf0a=B_Z_sf0a[i]*B_Z_sf0a[i]*T_ZZ_sf0a[i]    
        T_para_sf0a=((Sum_1_sf0a+Sum_2_sf0a+Sum_3_sf0a+Sum_4_sf0a+Sum_5_sf0a+Sum_6_sf0a+Sum_7_sf0a+Sum_8_sf0a+Sum_9_sf0a)/(B_mag_XYZ_sf0a[i])**2)
        Trace_Temp_sf0a=(T_XX_sf0a[i]+T_YY_sf0a[i]+T_ZZ_sf0a[i])
        T_perp_sf0a=(Trace_Temp_sf0a-T_para_sf0a)/2.0
        T_parallel_sf0a.append((Sum_1_sf0a+Sum_2_sf0a+Sum_3_sf0a+Sum_4_sf0a+Sum_5_sf0a+Sum_6_sf0a+Sum_7_sf0a+Sum_8_sf0a+Sum_9_sf0a)/(B_mag_XYZ_sf0a[i])**2)
        T_perpendicular_sf0a.append(T_perp_sf0a)
        Anisotropy_sf0a.append(T_perp_sf0a/T_para_sf0a)

    #Proton Analysis: Velocity, Energy Flux, and Temperature Anisotropy
    #==============================================================

    # This section performs the following operations on the proton data:
    # 1. Extracts and calculates velocity components (Vr, Vt, Vn) and magnitude
    # 2. Accesses proton energy flux data for energy, theta, and phi
    # 3. Extracts times, energy flux, and energy values
    # 4. Computes centroids for energy, theta, and phi
    # 5. Prepares time arrays for plotting
    # 6. Calculates temperature anisotropy from temperature tensor and magnetic field data
    # Note: Commented out code for interpolation to sf00 fit file cadence

    #Access components of velocity
    split_vec('psp_spi_sf0a_VEL_RTN_SUN') #split_vec saparates the three components out, we're in RTN so "x" becomes our "r" and so on...
    vr_sf0a_ = get_data('psp_spi_sf0a_VEL_RTN_SUN_x')
    vr_sf0a = vr_sf0a_.y
    vt_sf0a_ = get_data('psp_spi_sf0a_VEL_RTN_SUN_y')
    vt_sf0a = vt_sf0a_.y
    vn_sf0a_ = get_data('psp_spi_sf0a_VEL_RTN_SUN_z')
    vn_sf0a = vn_sf0a_.y
    vmag_sf0a = np.sqrt(vr_sf0a**2 + vt_sf0a **2 + vn_sf0a**2) #Here we're defining the magnitude of this vector, the data is stored in the .y component

    #In the future we'll create a function that computes the magnitude.

    #access proton energy flux vs energy -- For the ions we want to look at the energy, theta and phi planes...
    spi_sf0a_eflux_v_energy = get_data('psp_spi_sf0a_EFLUX_VS_ENERGY')
    spi_sf0a_eflux_v_theta = get_data('psp_spi_sf0a_EFLUX_VS_THETA')
    spi_sf0a_eflux_v_phi = get_data('psp_spi_sf0a_EFLUX_VS_PHI')

    spi_sf0a_times = spi_sf0a_eflux_v_energy.times #extracting the times

    spi_sf0a_nrg_flux = spi_sf0a_eflux_v_energy.y #energy flux vs energy, this is the data that will go into the color bar

    # Apply logarithmic transformation to the spi nrg flux vs nrg data
    log_spi_sf0a_nrg_flux = np.log10(spi_sf0a_nrg_flux)

    spi_sf0a_nrg_vals = spi_sf0a_eflux_v_energy.v #energy flux vs the actual energy values, this is the y axis. / The energy values for the  32 channels coppied at every time step (they do not change)

    #When we're looking at eflux vs e, at each time step we have the amount of flux and at what energy did we measure that flux.
    #Energy vs phi, we have the amount of flux and at which phi value did we measure that flux.

    spi_sf0a_nrg_flux_theta = spi_sf0a_eflux_v_theta.y
    log_spi_sf0a_nrg_flux_theta = np.log10(spi_sf0a_nrg_flux_theta)
    spi_sf0a_nrg_vals_theta = spi_sf0a_eflux_v_theta.v
    spi_sf0a_nrg_flux_phi = spi_sf0a_eflux_v_phi.y
    log_spi_sf0a_nrg_flux_phi = np.log10(spi_sf0a_nrg_flux_phi)
    spi_sf0a_nrg_vals_phi = spi_sf0a_eflux_v_phi.v

    datetime_spi_sf0a = time_datetime(spi_sf0a_times)
    print(datetime_spi_sf0a[0])
    print(datetime_spi_sf0a[-1])

    times_spi_sf0a_repeat = np.repeat(np.expand_dims(datetime_spi_sf0a,1),32,1)
    times_spi_sf0a_repeat_angle = np.repeat(np.expand_dims(datetime_spi_sf0a,1),8,1) 

    #compute centroids
    centroids_spi_sf0a_nrg = np.ma.average(spi_sf0a_nrg_vals, 
                           weights=spi_sf0a_nrg_flux, 
                           axis=1)
    centroids_spi_sf0a_theta = np.ma.average(spi_sf0a_nrg_vals_theta, 
                           weights=spi_sf0a_nrg_flux_theta, 
                           axis=1)
    centroids_spi_sf0a_phi = np.ma.average(spi_sf0a_nrg_vals_phi, 
                           weights=spi_sf0a_nrg_flux_phi, 
                           axis=1)


    # Proton Plasma Parameters Calculation
    #==============================================================
    # This cell calculates various plasma parameters for protons:
    # 1. Extracts plasma density and magnetic field data
    # 2. Calculates magnetic field magnitude
    # 3. Computes Alfvén speed
    # 4. Calculates parallel and perpendicular plasma beta
    # 5. Computes parallel and perpendicular plasma pressure
    # Note: All calculations use data from SPAN-Ion (spi_sf0a) instrument

    dens_spi_sf0a = get_data('psp_spi_sf0a_DENS') #the density of the plasma measured by span ion in units of cm^-3
    b_spi_sf0a = get_data('psp_spi_sf0a_MAGF_INST') #the magnetic field projected in the instrument frame at the same cadence as span ion

    # Calculate the magnitude of b_spi_sf0a
    b_mag_spi_sf0a = np.sqrt(np.sum(b_spi_sf0a.y**2, axis=1))

    #The alfven speed
    v_alfven_spi_sf0a = 21.8 * b_mag_spi_sf0a/ np.sqrt(dens_spi_sf0a.y) #alfven speed computed using spi_sf0a density


    # The Alfvén Mach number (alphas)
    M_alfven_spi_sf0a = vmag_sf0a / v_alfven_spi_sf0a

    #The Temperature
    temp_spi_sf0a = get_data('psp_spi_sf0a_TEMP')

    #The definition of beta is the particle pressure over the magnetic field pressure
    #p is for proton and this is the parallel and perpendicular beta
    beta_ppar_spi_sf0a = (4.03E-11*dens_spi_sf0a.y*T_parallel_sf0a)/(1e-5*b_mag_spi_sf0a)**2
    beta_pperp_spi_sf0a = (4.03E-11*dens_spi_sf0a.y*T_perpendicular_sf0a)/(1e-5*b_mag_spi_sf0a)**2
    beta_p_spi_sf0a = (4.03E-11*dens_spi_sf0a.y*temp_spi_sf0a.y)/(1e-5*b_mag_spi_sf0a)**2


    pressure_ppar_spi_sf0a = 1.602E-4*dens_spi_sf0a.y*T_parallel_sf0a #the original number was 1.602E-13, this is the converstion factor for electron vots per cubic CM to pascals, then we multiplied it by 1E9 (1*10^9) to convert to nanopascals 
    pressure_pperp_spi_sf0a = 1.602E-4*dens_spi_sf0a.y*T_perpendicular_sf0a
    pressure_p_spi_sf0a = 1.602E-4*temp_spi_sf0a.y*dens_spi_sf0a.y 
    #Pperp over ppar eventually

    # Calculate magnetic pressure using Bmag_spi_sf0a cadence (for tot pressure calc)
    mu_0 = constants.mu_0  # Permeability of free space
    pmag_spi_sf0a = (b_mag_spi_sf0a**2) / (2 * mu_0)
        # Convert to nPa (nanoPascals) to match your other pressure units
    pmag_spi_sf0a = pmag_spi_sf0a * 1e9
    #total pressure of magnetic plus proton (later add more terms like alpha and elec)
    pressure_tot_mag_p_sf0a = pressure_p_spi_sf0a + pmag_spi_sf0a


    #dostance from sun
    sun_dist_sf0a_ = get_data('psp_spi_sf0a_SUN_DIST')
    sun_dist_sf0a_km = sun_dist_sf0a_.y
    sun_dist_sf0a_rsun = sun_dist_sf0a_km/695700.0


In [ ]:
if (spi_sf00_mom and spi_sf0a_mom) is not None:
    divide('psp_spi_sf0a_DENS','psp_spi_sf00_DENS','Na/Np')
    na_div_np_ = get_data('Na/Np')
    na_div_np = na_div_np_.y
    
    #calculate alpha-proton drift speed as |Valphas|-|Vsw|. 
    store_data('|Vsw|',data={'x':vr_.times, 'y': v_sw_spi})
    store_data('|Valpha|',data={'x':vr_sf0a_.times, 'y': vmag_sf0a})
    
    #Note alphas are slower cadence than the protons, so need to interpolate first.
    tinterpol('|Vsw|','|Valpha|',newname = 'va-vp')
    
    vdrift_ap_ = get_data('va-vp')
    vdrift_ap = vdrift_ap_.y
    
    store_data('valfven_spi', data={'x':vr_.times, 'y': v_alfven_spi})
    #normalize by alfven speed
    divide('va-vp','valfven_spi','va-vp/vA')
    
    vdrift_ap_va_ = get_data('va-vp/vA')
    vdrift_ap_va = vdrift_ap_va_.y 
    

    #calculate Alpha/proton temperature ratio
    divide('psp_spi_sf0a_TEMP','psp_spi_sf00_TEMP','Ta/Tp')
    ta_div_tp_ = get_data('Ta/Tp')
    ta_div_tp = ta_div_tp_.y
    

In [ ]:
if wvpow is not None:
    wvpow_LH_dat = get_data('wavePower_LH')
    wvpow_times = wvpow_LH_dat.times
    datetime_wvpow = time_datetime(wvpow_times)
    wvpow_LH = wvpow_LH_dat.y
    wvpow_RH_dat = get_data('wavePower_RH')
    wvpow_RH = wvpow_RH_dat.y
    wvpow_LH_spi = downsample_time_based(datetime_wvpow, wvpow_LH, datetime_spi)
    wvpow_RH_spi = downsample_time_based(datetime_wvpow, wvpow_RH, datetime_spi)
    
    #histograms
    tau_30s = 30 #30 sec
    h_RH_30s, bin_centers_datetime_utc_RH_30s, bin_centers_vals_RH_30s=lista_hista2d(datetime_wvpow,np.log10(wvpow_RH),density=True, cmax=1e-3,tau=tau_30s, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Reds')
    h_LH_30s, bin_centers_datetime_utc_LH_30s, bin_centers_vals_LH_30s=lista_hista2d(datetime_wvpow,np.log10(wvpow_LH),density=True, cmax=1e-3,tau=tau_30s, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Blues')
    
    tau_2m = 2*60 #2 min
    h_RH_2m, bin_centers_datetime_utc_RH_2m, bin_centers_vals_RH_2m=lista_hista2d(datetime_wvpow,np.log10(wvpow_RH),density=True, cmax=1e-3,tau=tau_2m, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Reds')
    h_LH_2m, bin_centers_datetime_utc_LH_2m, bin_centers_vals_LH_2m=lista_hista2d(datetime_wvpow,np.log10(wvpow_LH),density=True, cmax=1e-3,tau=tau_2m, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Blues')
    
    tau_20m = 20*60 #20 min
    h_RH_20m, bin_centers_datetime_utc_RH_20m, bin_centers_vals_RH_20m=lista_hista2d(datetime_wvpow,np.log10(wvpow_RH),density=True, cmax=1e-3,tau=tau_20m, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Reds')
    h_LH_20m, bin_centers_datetime_utc_LH_20m, bin_centers_vals_LH_20m=lista_hista2d(datetime_wvpow,np.log10(wvpow_LH),density=True, cmax=1e-3,tau=tau_20m, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Blues')

    tau_90m = 90*60 #90 min
    h_RH_90m, bin_centers_datetime_utc_RH_90m, bin_centers_vals_RH_90m=lista_hista2d(datetime_wvpow,np.log10(wvpow_RH),density=True, cmax=1e-3,tau=tau_90m, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Reds')
    h_LH_90m, bin_centers_datetime_utc_LH_90m, bin_centers_vals_LH_90m=lista_hista2d(datetime_wvpow,np.log10(wvpow_LH),density=True, cmax=1e-3,tau=tau_90m, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Blues')

    tau_4h = 4*60*60 #4 hr
    h_RH_4h, bin_centers_datetime_utc_RH_4h, bin_centers_vals_RH_4h=lista_hista2d(datetime_wvpow,np.log10(wvpow_RH),density=True, cmax=1e-3,tau=tau_4h, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Reds')
    h_LH_4h, bin_centers_datetime_utc_LH_4h, bin_centers_vals_LH_4h=lista_hista2d(datetime_wvpow,np.log10(wvpow_LH),density=True, cmax=1e-3,tau=tau_4h, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Blues')

    tau_12h = 12*60*60 #12 hr
    h_RH_12h, bin_centers_datetime_utc_RH_12h, bin_centers_vals_RH_12h=lista_hista2d(datetime_wvpow,np.log10(wvpow_RH),density=True, cmax=1e-3,tau=tau_12h, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Reds')
    h_LH_12h, bin_centers_datetime_utc_LH_12h, bin_centers_vals_LH_12h=lista_hista2d(datetime_wvpow,np.log10(wvpow_LH),density=True, cmax=1e-3,tau=tau_12h, range = [[wvpow_times[0],wvpow_times[-1]],[-3,3]],cmap='Blues')

    # This creates a 2D array where each row corresponds to a timestamp and each column represents a pitch angle bin.
    times_RH_repeat_30s = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_RH_30s, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_30s.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    times_LH_repeat_30s = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_LH_30s, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_30s.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    
    times_RH_repeat_2m = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_RH_2m, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_2m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    times_LH_repeat_2m = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_LH_2m, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_2m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    
    times_RH_repeat_20m = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_RH_20m, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_20m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    times_LH_repeat_20m = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_LH_20m, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_20m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    # This creates a 2D array where each row corresponds to a timestamp and each column represents a pitch angle bin.
    times_RH_repeat_90m = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_RH_90m, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_90m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    times_LH_repeat_90m= np.repeat(
        np.expand_dims(bin_centers_datetime_utc_LH_90m, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_90m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    # This creates a 2D array where each row corresponds to a timestamp and each column represents a pitch angle bin.
    times_RH_repeat_4h = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_RH_4h, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_4h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    times_LH_repeat_4h = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_LH_4h, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_4h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    times_RH_repeat_12h = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_RH_12h, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_12h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    times_LH_repeat_12h = np.repeat(
        np.expand_dims(bin_centers_datetime_utc_LH_12h, 1),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_12h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 1                          # Repeat along axis=1 (columns)
    )
    
    ydat_RH_30s = np.power(10,bin_centers_vals_RH_30s)
    ydat_RH_repeat_30s = np.repeat(
        np.expand_dims(ydat_RH_30s, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_30s.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_LH_30s = np.power(10,bin_centers_vals_LH_30s)
    ydat_LH_repeat_30s = np.repeat(
        np.expand_dims(ydat_LH_30s, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_30s.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )   
    ydat_RH_2m = np.power(10,bin_centers_vals_RH_2m)
    ydat_RH_repeat_2m = np.repeat(
        np.expand_dims(ydat_RH_2m, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_2m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_LH_2m = np.power(10,bin_centers_vals_LH_2m)
    ydat_LH_repeat_2m = np.repeat(
        np.expand_dims(ydat_LH_2m, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_2m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )

    ydat_RH_20m = np.power(10,bin_centers_vals_RH_20m)
    ydat_RH_repeat_20m = np.repeat(
        np.expand_dims(ydat_RH_20m, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_20m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_LH_20m = np.power(10,bin_centers_vals_LH_20m)
    ydat_LH_repeat_20m = np.repeat(
        np.expand_dims(ydat_LH_20m, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_20m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_RH_90m = np.power(10,bin_centers_vals_RH_90m)
    ydat_RH_repeat_90m = np.repeat(
        np.expand_dims(ydat_RH_90m, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_90m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_LH_90m = np.power(10,bin_centers_vals_LH_90m)
    ydat_LH_repeat_90m = np.repeat(
        np.expand_dims(ydat_LH_90m, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_90m.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_RH_4h = np.power(10,bin_centers_vals_RH_4h)
    ydat_RH_repeat_4h = np.repeat(
        np.expand_dims(ydat_RH_4h, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_4h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_LH_4h = np.power(10,bin_centers_vals_LH_4h)
    ydat_LH_repeat_4h = np.repeat(
        np.expand_dims(ydat_LH_4h, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_4h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_RH_12h = np.power(10,bin_centers_vals_RH_12h)
    ydat_RH_repeat_12h = np.repeat(
        np.expand_dims(ydat_RH_12h, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_RH_12h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )
    ydat_LH_12h = np.power(10,bin_centers_vals_LH_12h)
    ydat_LH_repeat_12h = np.repeat(
        np.expand_dims(ydat_LH_12h, 0),  # Input array: expanded time array from (N,) to (N, 1) where N is the number of time steps
        h_LH_12h.shape[1],             # Number of repetitions (e.g. 12 pitch angle bins)
        axis = 0                          # Repeat along axis=1 (columns)
    )

    #compute centroids
    centroids_RH_30s = np.ma.average(ydat_RH_repeat_30s,axis=1, 
                        weights=h_RH_30s.T)
    centroids_LH_30s = np.ma.average(ydat_LH_repeat_30s,axis=1, 
                        weights=h_LH_30s.T)
    centroids_RH_2m = np.ma.average(ydat_RH_repeat_2m,axis=1, 
                        weights=h_RH_2m.T)
    centroids_LH_2m = np.ma.average(ydat_LH_repeat_2m,axis=1, 
                        weights=h_LH_2m.T)
    centroids_RH_20m = np.ma.average(ydat_RH_repeat_20m,axis=1, 
                        weights=h_RH_20m.T)
    centroids_LH_20m = np.ma.average(ydat_LH_repeat_20m,axis=1, 
                        weights=h_LH_20m.T)
    centroids_RH_90m = np.ma.average(ydat_RH_repeat_90m,axis=1, 
                        weights=h_RH_90m.T)
    centroids_LH_90m = np.ma.average(ydat_LH_repeat_90m,axis=1, 
                        weights=h_LH_90m.T)
    centroids_RH_4h = np.ma.average(ydat_RH_repeat_4h,axis=1, 
                        weights=h_RH_4h.T)
    centroids_LH_4h = np.ma.average(ydat_LH_repeat_4h,axis=1, 
                        weights=h_LH_4h.T)
    centroids_RH_12h = np.ma.average(ydat_RH_repeat_12h,axis=1, 
                        weights=h_RH_12h.T)
    centroids_LH_12h = np.ma.average(ydat_LH_repeat_12h,axis=1, 
                        weights=h_LH_12h.T)

In [ ]:
if sf00_fits is not None:
    filtered_val_index_sf00 = np.where(
        (df_sf00['np1'] > 10) &
        (df_sf00['np2'] > 10) &
        (df_sf00['Tperp1'] > .03) &
        (df_sf00['Tperp2'] > .03) &
        (df_sf00['Trat1'] > .01) &
        (df_sf00['Trat2'] > .01) &
        (df_sf00['Trat1'] != 30) &
        (df_sf00['Trat2'] != 30) &
        (df_sf00['Trat1'] != 2.0) &
        (df_sf00['Trat2'] != 2.0) &
        (df_sf00['Trat1'] != 1.0) &
        (df_sf00['Trat2'] != 1.0) &
        (df_sf00['vdrift'] != 10000) &
        (df_sf00['vdrift'] != -10000)
    )[0]
    
    # Build a mask of rows you want to KEEP
    mask_sf00 = df_sf00.index.isin(filtered_val_index_sf00)

    # Get just the data columns (everything except timestamp)
    columns_to_mask_sf00 = df_sf00.columns.difference(['time'])  # or whatever your time column is called

    # Mask only those columns
    df_sf00.loc[~mask_sf00, columns_to_mask_sf00] = np.nan

    # Create the 'datetime' column
    df_sf00['datetime'] = pd.to_datetime(df_sf00['time'], unit='s', utc=True)
    
    # Convert 'datetime' column to NumPy array of datetime.datetime objects
    datetime_array = df_sf00['datetime'].dt.to_pydatetime()
    
    if len(datetime_array) < len(datetime_spi):
        df_sf00 = upsample_to_match(df_sf00, datetime_array, datetime_spi)
        # Convert 'datetime' column to NumPy array of datetime.datetime objects
        datetime_array = df_sf00['datetime'].dt.to_pydatetime()
    
    #==============================================================
# sf00 fits
#==============================================================

    #Access components of velocity
    #split_vec('psp_spi_sf00_VEL_RTN_SUN')
   # vr = get_data('psp_spi_sf00_VEL_RTN_SUN_x')
   # vt = get_data('psp_spi_sf00_VEL_RTN_SUN_y')
   # vn = get_data('psp_spi_sf00_VEL_RTN_SUN_z')
   # vmag = np.sqrt(vr.y**2 + vt.y **2 + vn.y**2)

    df_sf00['vsw_mom'] = vmag #⭐️ The proton moments value needs to be calculatd for vsw_mach_pfits (see below)

    #calculate variables
    df_sf00['vp1_mag'] = np.sqrt(df_sf00['vp1_x']**2 + df_sf00['vp1_y']**2 + df_sf00['vp1_z']**2) 
    df_sf00['B_mag'] = np.sqrt(df_sf00['B_inst_x']**2 + df_sf00['B_inst_y']**2 + df_sf00['B_inst_z']**2)
    df_sf00['bhat_x'] = df_sf00['B_inst_x']/df_sf00['B_mag']
    df_sf00['bhat_y'] = df_sf00['B_inst_y']/df_sf00['B_mag']
    df_sf00['bhat_z'] = df_sf00['B_inst_z']/df_sf00['B_mag']
    m = 1836 * 511000.0 / (299792.0**2) #proton mass
    
    df_sf00['n_tot'] = df_sf00['np1'] + df_sf00['np2']
    df_sf00['Tperp_tot'] = (df_sf00['np1'] * df_sf00['Tperp1'] + df_sf00['np2'] * df_sf00['Tperp2'])/df_sf00['n_tot']
    df_sf00['Tpar1'] = df_sf00['Tperp1']/df_sf00['Trat1']
    df_sf00['Tpar2'] = df_sf00['Tperp2']/df_sf00['Trat2']
    df_sf00['Tpar_tot'] = (df_sf00['np1'] * df_sf00['Tpar1'] + df_sf00['np2'] * df_sf00['Tpar2'] + (df_sf00['np1'] * df_sf00['np2'] / df_sf00['n_tot'])* m * (df_sf00['vdrift']**2))/df_sf00['n_tot'] 
    df_sf00['Trat_tot'] = df_sf00['Tperp_tot']/df_sf00['Tpar_tot']
    df_sf00['Temp_tot'] = (2*df_sf00['Tperp_tot'] + df_sf00['Tpar_tot'])/3
    
    df_sf00['vp1_mag'] = np.sqrt(df_sf00['vp1_x']**2 + df_sf00['vp1_y']**2 + df_sf00['vp1_z']**2)
    df_sf00['vcm_x'] = df_sf00['vp1_x'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_x'] # fitted center of mass velocity (x)
    df_sf00['vcm_y'] = df_sf00['vp1_y'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_y'] # fitted center of mass velocity (y)
    df_sf00['vcm_z'] = df_sf00['vp1_z'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_z'] # fitted center of mass velocity (z)
    df_sf00['vcm_mag'] = np.sqrt(df_sf00['vcm_x']**2 + df_sf00['vcm_y']**2 + df_sf00['vcm_z']**2)

    df_sf00['vp2_x'] = df_sf00['vp1_x'] + df_sf00['vdrift'] * df_sf00['bhat_x'] # fitted center of p2 velocity (x)
    df_sf00['vp2_y'] = df_sf00['vp1_y'] + df_sf00['vdrift'] * df_sf00['bhat_y'] # fitted center of p2 velocity (y)
    df_sf00['vp2_z'] = df_sf00['vp1_z'] + df_sf00['vdrift'] * df_sf00['bhat_z'] # fitted center of p2 velocity (x)
    df_sf00['vp2_mag'] = np.sqrt(df_sf00['vp2_x']**2 + df_sf00['vp2_y']**2 + df_sf00['vp2_z']**2)

    df_sf00['vcm_y'] = df_sf00['vp1_y'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_y'] # fitted center of mass velocity (y)
    df_sf00['vcm_z'] = df_sf00['vp1_z'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_z'] # fitted center of mass velocity (z)
    df_sf00['vcm_mag'] = np.sqrt(df_sf00['vcm_x']**2 + df_sf00['vcm_y']**2 + df_sf00['vcm_z']**2)

    #compute heat flux
    m = 1836 * 511000.0 / (299792.0**2)

    #convert to thermal speeds
    vt1perp2 = 2 * df_sf00['Tperp1']/m 
    vt2perp2 = 2 * df_sf00['Tperp2']/m 
    vt1par2 = 2 * df_sf00['Tpar1']/m 
    vt2par2 = 2 * df_sf00['Tpar2']/m 

    fac = 1.602E-10 #W/m2 conversion
    df_sf00['qz_p'] = fac * 0.5 * m * ((df_sf00['np1'] * df_sf00['np2']) / df_sf00['n_tot']) * df_sf00['vdrift'] * (1.5 * (vt2par2 - vt1par2) + df_sf00['vdrift']**2 * (df_sf00['np1']**2 - df_sf00['np2']**2)/df_sf00['n_tot']**2 + (vt2perp2 - vt1perp2)) #⭐️ heat flux of the proton beam, in watts per meter squared, as blueviolet
    df_sf00['|qz_p|'] = np.abs(df_sf00['qz_p'])

    #compute normalisation factors

    df_sf00['vt_perp_tot'] = np.sqrt(2 * (df_sf00['np1'] * df_sf00['Tperp1'] + df_sf00['np2'] * df_sf00['Tperp2']) / (m * df_sf00['n_tot']))
    df_sf00['vt_par_tot'] = np.sqrt(2 * (df_sf00['np1'] * df_sf00['Tpar1']+ df_sf00['np2'] * df_sf00['Tpar2'] + m * df_sf00['vdrift']**2 * (df_sf00['np1'] * df_sf00['np2'])/df_sf00['n_tot']) / (m * df_sf00['n_tot']))

    df_sf00['qz_p_perp']= df_sf00['qz_p'] / (m * df_sf00['n_tot'] * df_sf00['vt_perp_tot']**3) # also normalise by mass
    df_sf00['qz_p_par'] = df_sf00['qz_p'] / (m * df_sf00['n_tot'] * df_sf00['vt_par_tot']**3)

#Compute Alfven Mach number and Proton Parallel, Perp Beta

#calculate Alfven speed
    df_sf00['valfven_pfits'] = 21.8 * df_sf00['B_mag']/ np.sqrt(df_sf00['n_tot']) #using proton p1 + p2 total density

#defining normalized drift speed using fitted density for vA
    df_sf00['vdrift_va_pfits'] = df_sf00['vdrift']/df_sf00['valfven_pfits'] 
    #df_sf00['|valfven_va_pfits|'] = np.abs(df_sf00['valfven_va_pfits'])
    #df_sf00['KE_p2'] = 0.5 * df_sf00['|valfven_pfits|']**2

#defining center of mass velocity magnitude as |Vsw| compute Vsw mach
    df_sf00['Vcm_mach_pfits'] = df_sf00['vcm_mag']/df_sf00['valfven_pfits'] #using valfven from fits
#defining p1 velocity magnitude as |Vsw| compute Vsw mach
    df_sf00['Vp1_mach_pfits'] = df_sf00['vp1_mag']/df_sf00['valfven_pfits'] #using valfven from fits
#defining moment velocity magnitude as |Vsw| compute Vsw mach
    df_sf00['vsw_mach_pfits'] = df_sf00['vsw_mom']/df_sf00['valfven_pfits'] #using valfven from fits ⭐️ Taking the solar window value from the moments and normalizing by the alfven speed, using fitted density as the density for alfven speed (Moments interpoalted to the same time cadence as the data file and added to the data frame.)

#compute proton beta 
#note 1e-5 is factor conversion from nT to cgs
    df_sf00['beta_ppar_pfits'] = (4.03E-11 * df_sf00['n_tot'] * df_sf00['Tpar_tot'])/(1e-5 * df_sf00['B_mag'])**2 #total BetaPar from pfit dens ⭐️ 
    df_sf00['beta_pperp_pfits'] = (4.03E-11 * df_sf00['n_tot'] * df_sf00['Tperp_tot'])/(1e-5 * df_sf00['B_mag'])**2 #total BetaPerp from pfit dens ⭐️ 
    df_sf00['beta_par_p1'] = (4.03E-11 * df_sf00['np1'] * df_sf00['Tpar1'])/(1e-5 * df_sf00['B_mag'])**2 #proton p1 BetaPar
    df_sf00['beta_par_p2'] = (4.03E-11 * df_sf00['np2'] * df_sf00['Tpar2'])/(1e-5 * df_sf00['B_mag'])**2 #proton p2 BetaPar
    df_sf00['beta_perp_p1'] = (4.03E-11 * df_sf00['np1'] * df_sf00['Tperp1'])/(1e-5 * df_sf00['B_mag'])**2 #proton p1 BetaPerp
    df_sf00['beta_perp_p2'] = (4.03E-11 * df_sf00['np2'] * df_sf00['Tperp2'])/(1e-5 * df_sf00['B_mag'])**2 #proton p2 BetaPerp

    df_sf00['beta_p_pfits_tot'] = np.sqrt(df_sf00['beta_ppar_pfits']**2 + df_sf00['beta_pperp_pfits']**2)

#hammerhead diagnostic
    df_sf00['ham_param'] = (df_sf00['Tperp2']/df_sf00['Tperp1'])/df_sf00['Trat_tot'] # ⭐️ Preliminary proxy as a value for the hamplitude, hammer headness. Ratio of the perp temp of the beam to the perp temp of the core, how much hotter the beam is over the core, all that normalized over the total temp anisot of the core and beam. Palevioletred color

#--------------------FITS Variables For Plotbot--------------------\\
##-----------------------------------------------------------------||
#Converting from pandas data frames to numpy arrays
    qz_p = df_sf00['qz_p'].to_numpy() # Heat flux of the proton beam, in watts per meter squared, as blueviolet
    vsw_mach_pfits = df_sf00['vsw_mach_pfits'].to_numpy() # Using valfven from fits. Taking the solar window value from the moments and normalizing by the alfven speed, using fitted density as the density for alfven speed (Moments interpoalted to the same time cadence as the data file and added to the data frame.)
    beta_ppar_pfits = df_sf00['beta_ppar_pfits'].to_numpy()  # Total Proton BetaPar from pfit dens. Hot pink color
    beta_pperp_pfits = df_sf00['beta_pperp_pfits'].to_numpy() # Total Proton BetaPerp from pfit dens. Baby blue color
    ham_param = df_sf00['ham_param'].to_numpy() # Preliminary proxy as a value for the hamplitude, hammer headness. Ratio of the perp temp of the beam to the perp temp of the core, how much hotter the beam is over the core, all that normalized over the total temp anisot of the core and beam. Palevioletred color
    
    np1 = df_sf00['np1'].to_numpy() # Core density, marker (5,1) makes a star. He likes hot pink and deep skye blue
    np2 = df_sf00['np2'].to_numpy() # Beam density -- for s=s, s defines the size of the point, alpha defines the tranparency of the point. When looking at a full day, s 5-10 and trasparency .7, zooming in then s 200 and transparency to be 1 (adap the size dynamically, adjust the size of the symbol based on the number of points plotted).
    n_tot = df_sf00['n_tot'].to_numpy() #total beam + core dens
    
    vp1_x = df_sf00['vp1_x'].to_numpy() # Core velocity vx
    vp1_y = df_sf00['vp1_y'].to_numpy() # Core velocity vy
    vp1_z = df_sf00['vp1_z'].to_numpy() # Core velocity vz
    vp1_mag = df_sf00['vp1_mag'].to_numpy() # Core velocity vmag
    vcm_x = df_sf00['vcm_x'].to_numpy() #fitted center of mass vx
    vcm_y = df_sf00['vcm_y'].to_numpy() #fitted center of mass vy
    vcm_z = df_sf00['vcm_z'].to_numpy() #fitted center of mass vz
    vcm_mag = df_sf00['vcm_mag'].to_numpy() #fitted center of mass vmag
    vdrift = df_sf00['vdrift'].to_numpy() # Drift speed 
    vdrift_va_pfits = df_sf00['vdrift_va_pfits'].to_numpy() # Drift speed normalized by the alfven speed. It's useful to have both variables available.
    
    Trat1 = df_sf00['Trat1'].to_numpy() # Temp anisotropy of the core
    Trat2 = df_sf00['Trat2'].to_numpy() # Temp anisotropy of the beam
    Trat_tot = df_sf00['Trat_tot'].to_numpy() # Temp anisotropy of the combined core and beam
    Tperp1 = df_sf00['Tperp1'].to_numpy() # Temp perp of the core
    Tperp2 = df_sf00['Tperp2'].to_numpy() # Temp perp of the beam
    Tperp_tot = df_sf00['Tperp_tot'].to_numpy() # Temp perp of the combined core and beam
    Tpar1 = df_sf00['Tpar1'].to_numpy() # Temp par of the core
    Tpar2 = df_sf00['Tpar2'].to_numpy() # Temp par of the beam
    Tpar_tot = df_sf00['Tpar_tot'].to_numpy() # Temp par of the combined core and beam
    Temp_tot = df_sf00['Temp_tot'].to_numpy() # Temp of the combined core and beam

    abs_qz_p = df_sf00['|qz_p|'].to_numpy()
    
    #uncertainties
    np1_dpar = df_sf00['np1_dpar'].to_numpy() # Core density, marker (5,1) makes a star. He likes hot pink and deep skye blue
    np2_dpar = df_sf00['np2_dpar'].to_numpy() # Beam density
    vp1_x_dpar = df_sf00['vp1_x_dpar'].to_numpy() # Core velocity vx
    vp1_y_dpar = df_sf00['vp1_y_dpar'].to_numpy() # Core velocity vy
    vp1_z_dpar = df_sf00['vp1_z_dpar'].to_numpy() # Core velocity vz
    vp1_mag_dpar = np.sqrt(vp1_x_dpar**2 + vp1_y_dpar**2 + vp1_z_dpar**2) 
    vdrift_dpar = df_sf00['vdrift_dpar'].to_numpy() # Drift speed
    Trat1_dpar = df_sf00['Trat1_dpar'].to_numpy() # Temp anisotropy of the core
    Trat2_dpar = df_sf00['Trat2_dpar'].to_numpy() # Temp anisotropy of the beam
    Tperp1_dpar = df_sf00['Tperp1_dpar'].to_numpy() # Temp perp of the core
    Tperp2_dpar = df_sf00['Tperp2_dpar'].to_numpy() # Temp perp of the beam
    chi_p = df_sf00['chi'].to_numpy() # chi (overall fit)


In [ ]:
if sf01_fits is not None:
    filtered_val_index_sf01 = np.where(
        (df_sf01['na'] > 0) & 
        (df_sf01['Ta_perp'] > .101) & 
        (df_sf01['Trata'] > .101) & 
        (df_sf01['Trata'] != 8) & 
        (df_sf01['Trata'] != 2.0)
        )[0]
        
    # Build a mask of rows you want to KEEP
    mask_sf01 = df_sf01.index.isin(filtered_val_index_sf01)

    # Get just the data columns (everything except timestamp)
    columns_to_mask_sf01 = df_sf01.columns.difference(['time'])  # or whatever your time column is called

    # Mask only those columns
    df_sf01.loc[~mask_sf01, columns_to_mask_sf01] = np.nan
        
    # Create the 'datetime' column
    df_sf01['datetime'] = pd.to_datetime(df_sf01['time'], unit='s', utc=True)

    # Convert 'datetime' column to NumPy array of datetime.datetime objects
    datetime_array_sf01 = df_sf01['datetime'].dt.to_pydatetime()
    
    if len(datetime_array_sf01) < len(datetime_spi_sf0a):
        #upsample fits file to match sf0a moment cadence
        df_sf01 = upsample_to_match(df_sf01, datetime_array_sf01, datetime_spi_sf0a)
        # Convert 'datetime' column to NumPy array of datetime.datetime objects
        datetime_array_sf01 = df_sf01['datetime'].dt.to_pydatetime()
    
    #if sf01 has lower cadence than sf00, 
    #downsample sf00 quantities for math on sf01
    if len(datetime_array_sf01) < len(datetime_array):
        vp1_mag_ = downsample_to_match(datetime_array, vp1_mag, datetime_array_sf01)
        np1_ = downsample_to_match(datetime_array, np1, datetime_array_sf01)
        np2_ = downsample_to_match(datetime_array, np2, datetime_array_sf01)
    else:
        vp1_mag_ = vp1_mag
        np1_ = np1
        np2_ = np2

    np_tot_ = np1_ + np2_
    df_sf01['nap_tot'] = np_tot_ + df_sf01['na']
    df_sf01['B_mag'] = np.sqrt(df_sf01['B_inst_x']**2 + df_sf01['B_inst_y']**2 + df_sf01['B_inst_z']**2)
    df_sf01['valfven_apfits'] = 21.8 * df_sf01['B_mag']/ np.sqrt(df_sf01['nap_tot']) #using proton and alpha p1 + p2 + a  total density
    
    #if sf01 has lower cadence than sf00, upsample sf01 quantities for math on sf00
    if len(datetime_array_sf01) < len(datetime_array):
        valfven_apfits_ = upsample_to_match(df_sf01['valfven_apfits'].to_numpy(), datetime_array_sf01, datetime_array)
    else:
        valfven_apfits_ = df_sf01['valfven_apfits']
    df_sf00['valfven_apfits'] = valfven_apfits_
    #redefining normalized drift speed using fitted alpha and proton density for vA
    df_sf00['vdrift_va_apfits'] = df_sf00['vdrift']/df_sf00['valfven_apfits'] 

    df_sf01['va_mag'] = np.sqrt(df_sf01['va_x']**2 + df_sf01['va_y']**2 + df_sf01['va_z']**2)
    df_sf01['vdrift_ap1'] = df_sf01['va_mag'] - vp1_mag_
    df_sf01['vdrift_va_ap1'] = df_sf01['vdrift_ap1']/df_sf01['valfven_apfits']
    df_sf01['na/np_tot'] = df_sf01['na']/np_tot_
    df_sf01['na/np1'] = df_sf01['na']/np1_
    df_sf01['na/np2'] = df_sf01['na']/np2_
    df_sf01['Tpara'] = df_sf01['Ta_perp']/df_sf01['Trata']
    df_sf01['beta_par_a'] = (4.03E-11 * df_sf01['na'] * df_sf01['Tpara'])/(1e-5 * df_sf01['B_mag'])**2 #alpha BetaPar
    
    na = df_sf01['na'].to_numpy()
    na_div_nptot = df_sf01['na/np_tot'].to_numpy()
    na_div_np1 = df_sf01['na/np1'].to_numpy()
    na_div_np2 = df_sf01['na/np2'].to_numpy()
    va_x = df_sf01['va_x'].to_numpy()
    va_y = df_sf01['va_y'].to_numpy()
    va_z = df_sf01['va_z'].to_numpy()
    va_z = df_sf01['va_z'].to_numpy()
    va_mag = df_sf01['va_mag'].to_numpy()
    vdrift_ap1 = df_sf01['vdrift_ap1'].to_numpy()
    vdrift_va_ap1 = df_sf01['vdrift_va_ap1'].to_numpy()
    vdrift_va_apfits = df_sf00['vdrift_va_apfits'].to_numpy()
    Trata = df_sf01['Trata'].to_numpy()
    Tperpa = df_sf01['Ta_perp'].to_numpy()
    Tpara = df_sf01['Tpara'].to_numpy()
    beta_par_a = df_sf01['beta_par_a'].to_numpy()
    
    #uncertainties
    na_dpar = df_sf01['na_dpar'].to_numpy() # Core density, marker (5,1) makes a star. He likes hot pink and deep skye blue
    va_x_dpar = df_sf01['va_x_dpar'].to_numpy() # Core velocity vx
    va_y_dpar = df_sf01['va_y_dpar'].to_numpy() # Core velocity vy
    va_z_dpar = df_sf01['va_z_dpar'].to_numpy() # Core velocity vz
    va_mag_dpar = np.sqrt(va_x_dpar**2 + va_y_dpar**2 + va_z_dpar**2) 
    Trata_dpar = df_sf01['Trata_dpar'].to_numpy() # Temp anisotropy of the core
    Tperpa_dpar = df_sf01['Ta_perp_dpar'].to_numpy() # Temp perp of the core
    chi_a = df_sf01['chi'].to_numpy() # chi (overall fit)

In [ ]:
if ham is not None:
    hardham_list = []
    core_dens_list = []
    neck_dens_list = []
    ham_dens_list = []
    core_ux_list = []
    core_uy_list = []
    core_uz_list = []
    neck_ux_list = []
    neck_uy_list = []
    neck_uz_list = []
    ham_ux_list = []
    ham_uy_list = []
    ham_uz_list = []
    core_Txx_list = []
    core_Txy_list = []
    core_Txz_list = []
    core_Tyy_list = []
    core_Tyz_list = []
    core_Tzz_list = []
    neck_Txx_list = []
    neck_Txy_list = []
    neck_Txz_list = []
    neck_Tyy_list = []
    neck_Tyz_list = []
    neck_Tzz_list = []
    ham_Txx_list = []
    ham_Txy_list = []
    ham_Txz_list = []
    ham_Tyy_list = []
    ham_Tyz_list = []
    ham_Tzz_list = []
    ogflag_list = []

    for key in datham.keys():
        try:
            core_dens_list.append(datham[key]['core_moments']['n'])
            neck_dens_list.append(datham[key]['neck_moments']['n'])
            ham_dens_list.append(datham[key]['hammer_moments']['n'])

            core_ux_list.append(datham[key]['core_moments']['Ux'])
            core_uy_list.append(datham[key]['core_moments']['Uy'])
            core_uz_list.append(datham[key]['core_moments']['Uz'])

            neck_ux_list.append(datham[key]['neck_moments']['Ux'])
            neck_uy_list.append(datham[key]['neck_moments']['Uy'])
            neck_uz_list.append(datham[key]['neck_moments']['Uz'])

            ham_ux_list.append(datham[key]['hammer_moments']['Ux'])
            ham_uy_list.append(datham[key]['hammer_moments']['Uy'])
            ham_uz_list.append(datham[key]['hammer_moments']['Uz'])

            core_Txx_list.append(datham[key]['core_moments']['Txx'])
            core_Txy_list.append(datham[key]['core_moments']['Txy'])
            core_Txz_list.append(datham[key]['core_moments']['Txz'])
            core_Tyy_list.append(datham[key]['core_moments']['Tyy'])
            core_Tyz_list.append(datham[key]['core_moments']['Tyz'])
            core_Tzz_list.append(datham[key]['core_moments']['Tzz'])

            neck_Txx_list.append(datham[key]['neck_moments']['Txx'])
            neck_Txy_list.append(datham[key]['neck_moments']['Txy'])
            neck_Txz_list.append(datham[key]['neck_moments']['Txz'])
            neck_Tyy_list.append(datham[key]['neck_moments']['Tyy'])
            neck_Tyz_list.append(datham[key]['neck_moments']['Tyz'])
            neck_Tzz_list.append(datham[key]['neck_moments']['Tzz'])

            ham_Txx_list.append(datham[key]['hammer_moments']['Txx'])
            ham_Txy_list.append(datham[key]['hammer_moments']['Txy'])
            ham_Txz_list.append(datham[key]['hammer_moments']['Txz'])
            ham_Tyy_list.append(datham[key]['hammer_moments']['Tyy'])
            ham_Tyz_list.append(datham[key]['hammer_moments']['Tyz'])
            ham_Tzz_list.append(datham[key]['hammer_moments']['Tzz'])

            # print(datham[key]['og_flag'])
            try: ogflag_list.append(datham[key]['og_flag'])
            except: ogflag_list.append(False)
            hardham_list.append(key)

        except: pass

    ogflag_list = np.asarray(ogflag_list)
    hardham_list = np.asarray(hardham_list)
    hamstring = np.ones_like(hardham_list)
    from datetime import datetime, timedelta, timezone
    hardham_datetime_cdflib = cdflib.cdfepoch.to_datetime(hardham_list)

    # Convert to pandas Timestamps (list comprehension)
    hardham_datetime = [pd.Timestamp(t) for t in hardham_datetime_cdflib]
    # Now timestamps.replace() will work!
    # Converting the magnetic field time to a timezone-aware datetime object
    hardham_datetime_utc = np.array([dt.replace(tzinfo=timezone.utc) for dt in hardham_datetime])
    #hardham_dattime = np.datetime64(hardham_datetime)

    dt_hardham_datetime_utc = np.diff(np.asarray(hardham_datetime_utc))
    dt_hamstring = np.ones_like(dt_hardham_datetime_utc)
    hardham_datetime_utc_plus_dt = []
    for dt in range(len(dt_hardham_datetime_utc)):
        hardham_datetime_utc_plus_dt.append(hardham_datetime_utc[dt] + dt_hardham_datetime_utc[dt])

    hardham_datetime_utc_plus_dt = np.asarray(hardham_datetime_utc_plus_dt)    
    hamstring_plus_dt = np.ones_like(hardham_datetime_utc_plus_dt)

    core_dens_list = np.asarray(core_dens_list)
    neck_dens_list = np.asarray(neck_dens_list)
    ham_dens_list = np.asarray(ham_dens_list)

    core_ux_list = np.asarray(core_ux_list)
    core_uy_list = np.asarray(core_uy_list)
    core_uz_list = np.asarray(core_uz_list)
    core_umag = np.sqrt(core_ux_list**2 + core_uy_list**2 + core_uz_list**2)

    neck_ux_list = np.asarray(neck_ux_list)
    neck_uy_list = np.asarray(neck_uy_list)
    neck_uz_list = np.asarray(neck_uz_list)
    neck_umag = np.sqrt(neck_ux_list**2 + neck_uy_list**2 + neck_uz_list**2)
    neck_core_drift = abs(neck_umag - core_umag)

    ham_ux_list = np.asarray(ham_ux_list)
    ham_uy_list = np.asarray(ham_uy_list)
    ham_uz_list = np.asarray(ham_uz_list)
    ham_umag = np.sqrt(ham_ux_list**2 + ham_uy_list**2 + ham_uz_list**2)
    ham_core_drift = np.abs(ham_umag - core_umag)
    
    b_ds = downsample_to_min_ind(datetime_spi, B_mag_XYZ, hardham_datetime_utc)
    d_ds = downsample_to_min_ind(datetime_spi, dens_spi.y, hardham_datetime_utc)
    #The alfven speed
    v_alfven_spi_ham = 21.8 * b_ds/ np.sqrt(d_ds)
    ham_core_drift_va = ham_core_drift/v_alfven_spi_ham
    neck_core_drift_va = neck_core_drift/v_alfven_spi_ham

    conv_fac = 1e6/11600 #MK to eV
    conv_fac = 1/conv_fac

    core_Txx_list = np.asarray(core_Txx_list)/conv_fac
    core_Txy_list = np.asarray(core_Txy_list)/conv_fac
    core_Txz_list = np.asarray(core_Txz_list)/conv_fac
    core_Tyy_list = np.asarray(core_Tyy_list)/conv_fac
    core_Tyz_list = np.asarray(core_Tyz_list)/conv_fac
    core_Tzz_list = np.asarray(core_Tzz_list)/conv_fac

    core_temp = (core_Txx_list + core_Tyy_list + core_Tyy_list)/3  

    neck_Txx_list = np.asarray(neck_Txx_list)/conv_fac
    neck_Txy_list = np.asarray(neck_Txy_list)/conv_fac
    neck_Txz_list = np.asarray(neck_Txz_list)/conv_fac
    neck_Tyy_list = np.asarray(neck_Tyy_list)/conv_fac
    neck_Tyz_list = np.asarray(neck_Tyz_list)/conv_fac
    neck_Tzz_list = np.asarray(neck_Tzz_list)/conv_fac

    neck_temp = (neck_Txx_list + neck_Tyy_list + neck_Tyy_list)/3  

    ham_Txx_list = np.asarray(ham_Txx_list)/conv_fac
    ham_Txy_list = np.asarray(ham_Txy_list)/conv_fac
    ham_Txz_list = np.asarray(ham_Txz_list)/conv_fac
    ham_Tyy_list = np.asarray(ham_Tyy_list)/conv_fac
    ham_Tyz_list = np.asarray(ham_Tyz_list)/conv_fac
    ham_Tzz_list = np.asarray(ham_Tzz_list)/conv_fac

    ham_temp = (ham_Txx_list + ham_Tyy_list + ham_Tyy_list)/3  

    spi_epoch_dat = get_data('psp_spi_sf00_Epoch')
    spi_epoch = spi_epoch_dat.y

    T_perp_core, T_parallel_core, Anisotropy_core = find_Tanisotropy(core_Txx_list,core_Tyy_list,core_Tzz_list,
                                                                     core_Txy_list,core_Txz_list,core_Tyz_list)
    T_perp_neck, T_parallel_neck, Anisotropy_neck = find_Tanisotropy(neck_Txx_list,neck_Tyy_list,neck_Tzz_list,
                                                                     neck_Txy_list,neck_Txz_list,neck_Tyz_list)
    T_perp_ham, T_parallel_ham, Anisotropy_ham = find_Tanisotropy(ham_Txx_list,ham_Tyy_list,ham_Tzz_list,
                                                                  ham_Txy_list,ham_Txz_list,ham_Tyz_list)

    Ntot = core_dens_list + neck_dens_list + ham_dens_list
    Nham_div_Ntot = ham_dens_list/Ntot
    Nham_div_Ncore = ham_dens_list/core_dens_list
    Nneck_div_Ncore = neck_dens_list/core_dens_list
    
    Tperp_ham_div_core = T_perp_ham/T_perp_core
    Tperprat_driftva_hc = Tperp_ham_div_core * ham_core_drift_va

    tau_30s = 30 #30 sec
    c_30s, bin_centers_datetime_ham_30s =lista_hista(hardham_datetime, tau_30s) 
    #hardham_datetime_og = hardham_datetime[ogflag_list]
    c_og_30s, bin_centers_datetime_ham_og_30s =lista_hista(np.asarray(hardham_datetime)[ogflag_list], tau_30s)
    
    tau_1m = 1*60 #1 min
    c_1m, bin_centers_datetime_ham_1m =lista_hista(hardham_datetime, tau_1m) 
    #hardham_datetime_og = hardham_datetime[ogflag_list]
    c_og_1m, bin_centers_datetime_ham_og_1m =lista_hista(np.asarray(hardham_datetime)[ogflag_list], tau_1m) 
    
    tau_2m = 2*60 #2 min
    c_2m, bin_centers_datetime_ham_2m =lista_hista(hardham_datetime, tau_2m) 
    #hardham_datetime_og = hardham_datetime[ogflag_list]
    c_og_2m, bin_centers_datetime_ham_og_2m =lista_hista(np.asarray(hardham_datetime)[ogflag_list], tau_2m) 
    
    tau_20m = 20*60 #20 min
    c_20m, bin_centers_datetime_ham_20m =lista_hista(hardham_datetime, tau_20m) 
    #hardham_datetime_og = hardham_datetime[ogflag_list]
    c_og_20m, bin_centers_datetime_ham_og_20m =lista_hista(np.asarray(hardham_datetime)[ogflag_list], tau_20m) 

    tau_90m = 90*60 #90 min
    c_90m, bin_centers_datetime_ham_90m =lista_hista(hardham_datetime, tau_90m) 
    #hardham_datetime_og = hardham_datetime[ogflag_list]
    c_og_90m, bin_centers_datetime_ham_og_90m =lista_hista(np.asarray(hardham_datetime)[ogflag_list], tau_90m)

    tau_4h = 4*60*60 #4 hr
    c_4h, bin_centers_datetime_ham_4h =lista_hista(hardham_datetime, tau_4h) 
    #hardham_datetime_og = hardham_datetime[ogflag_list]
    c_og_4h, bin_centers_datetime_ham_og_4h =lista_hista(np.asarray(hardham_datetime)[ogflag_list], tau_4h)

    tau_12h = 12*60*60 #12 hr
    c_12h, bin_centers_datetime_ham_12h =lista_hista(hardham_datetime, tau_12h) 
    #hardham_datetime_og = hardham_datetime[ogflag_list]
    c_og_12h, bin_centers_datetime_ham_og_12h =lista_hista(np.asarray(hardham_datetime)[ogflag_list], tau_12h)

    
    
    #upsample to sf00 moment timegrid
    c_30s_up = upsample_to_match(bin_centers_datetime_ham_30s,c_30s, datetime_spi)
    c_og_30s_up = upsample_to_match(bin_centers_datetime_ham_og_30s,c_og_30s, datetime_spi)
    c_2m_up = upsample_to_match(bin_centers_datetime_ham_2m,c_2m, datetime_spi)
    c_og_2m_up = upsample_to_match(bin_centers_datetime_ham_og_2m,c_og_2m, datetime_spi)
    c_20m_up = upsample_to_match(bin_centers_datetime_ham_20m,c_20m, datetime_spi)
    c_og_20m_up = upsample_to_match(bin_centers_datetime_ham_og_20m,c_og_20m, datetime_spi)
    c_90m_up = upsample_to_match(bin_centers_datetime_ham_90m,c_90m, datetime_spi)
    c_og_90m_up = upsample_to_match(bin_centers_datetime_ham_og_90m,c_og_90m, datetime_spi)
    c_4h_up = upsample_to_match(bin_centers_datetime_ham_4h,c_4h, datetime_spi)
    c_og_4h_up = upsample_to_match(bin_centers_datetime_ham_og_4h,c_og_4h, datetime_spi)
    c_12h_up = upsample_to_match(bin_centers_datetime_ham_12h,c_12h, datetime_spi)
    c_og_12h_up = upsample_to_match(bin_centers_datetime_ham_og_12h,c_og_12h, datetime_spi)
    
    
    
    Anisotropy_ham_up = upsample_to_match(hardham_datetime_utc,Anisotropy_ham, datetime_spi)
    ham_core_drift_up = upsample_to_match(hardham_datetime_utc,ham_core_drift, datetime_spi)
    ham_core_drift_va_up = upsample_to_match(hardham_datetime_utc,ham_core_drift_va, datetime_spi)
    Nham_div_Ncore_up = upsample_to_match(hardham_datetime_utc,Nham_div_Ncore, datetime_spi)
    Nham_div_Ntot_up = upsample_to_match(hardham_datetime_utc,Nham_div_Ntot, datetime_spi)
    Tperp_ham_div_core_up = upsample_to_match(hardham_datetime_utc,Tperp_ham_div_core, datetime_spi)
    Tperprat_driftva_hc_up = upsample_to_match(hardham_datetime_utc,Tperprat_driftva_hc, datetime_spi)